This script is for converting the matsim output files (generated by Aurore) into street graphs

In [40]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import shapely as shp
import pyproj

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'SNMan','SNMan Shared','data_v2'
)
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
paper_path = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'E-Bike City Accessibility','EBC Accessibility Paper - Shared'
)
matsim_results_path = os.path.join(
    paper_path, 'MATSim results', '2024-07-12 Travel times before and after'
)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [41]:
TIME = '18:00'
STATE = 'before'

In [21]:
# read the street graph
G = snman.io.load_street_graph(
    os.path.join(data_directory, 'outputs', '_paper2_example', 'G_edges.gpkg'),
    os.path.join(data_directory, 'outputs', '_paper2_example', 'G_nodes.gpkg'),
    crs=CRS_internal
)

In [49]:
# read the street graph
G = snman.io.load_street_graph(
    os.path.join(outputs_path, 'G_edges.gzip'),
    os.path.join(outputs_path, 'G_nodes.gzip'),
    crs=CRS_internal
)

In [42]:
# read the original osm export
osm_export = snman.io.import_geofile_to_gdf(
    os.path.join(matsim_results_path, f'{STATE}_oneway_links_exploded.gpkg')
)

In [43]:
# read the matsim output
tt = pd.read_csv(
    os.path.join(matsim_results_path, f'{STATE}_bike100pct.csv')
)
tt.set_index('OSM_ID', inplace=True)
tt

,LinkId,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,car_0:30,...,bike_21:30,car_22:00,bike_22:00,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104
OSM_ID,,,,,,,,,,,,,,,,,,,,,
7749021,3640,15.39,25.64,47.327088,8.381223,47.326734,8.380054,15.39,25.64,15.39,...,25.64,22.00,25.64,22.00,25.64,22.00,25.64,22.00,25.64,NaN
7701086,3638,22.48,22.48,47.499187,8.543563,47.499622,8.544628,22.48,22.48,22.48,...,22.48,28.00,22.48,22.48,22.48,22.48,22.48,22.48,22.48,NaN
7701087,3639,11.96,11.96,47.433009,8.668959,47.432571,8.669096,11.96,11.96,11.96,...,11.96,11.96,11.96,11.96,18.00,11.96,11.96,11.96,11.96,NaN
7701079,3630,40.03,40.03,47.490902,8.556413,47.492244,8.556424,40.03,40.03,40.03,...,40.03,46.00,40.03,40.03,40.03,40.03,40.03,46.00,40.03,NaN
7642172,3633,4.14,13.81,47.312737,8.913163,47.312457,8.913803,4.14,13.81,4.14,...,13.81,4.14,13.81,4.14,13.81,4.14,13.81,4.14,13.81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7694151,36256,30.34,30.34,47.536786,8.728047,47.536773,8.729726,30.34,30.34,30.34,...,36.00,30.34,30.34,30.34,30.34,30.34,30.34,30.34,30.34,NaN
7822078,132450,19.23,19.23,47.395599,8.401946,47.396158,8.401275,19.23,19.23,19.23,...,19.23,26.00,19.23,26.00,19.23,19.23,19.23,25.50,19.23,NaN
7740686,3629,46.18,46.18,47.349966,7.987588,47.351453,7.988677,46.18,46.18,46.18,...,46.18,53.00,46.18,46.18,46.18,53.00,46.18,53.00,46.18,NaN


In [44]:
osm_export['osm_id'] = osm_export['osm_id'].astype('int64')
osm_export['uvk'] = osm_export.apply(
    lambda row: (row['_u'], row['_v'], row['_key']),
    axis=1
)
#osm_export.set_index('uvk', inplace=True)
osm_export

,osm_id,name,highway,waterway,aerialway,barrier,man_made,railway,z_order,other_tags,...,grade,cost_private_cars_>,cost_private_cars_<,cost_cycling_>,cost_cycling_<,_v,_u,_key,geometry,uvk
0,7550200,NaN,residential,NaN,NaN,NaN,NaN,NaN,3,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,0.007,145.089,inf,145.089,inf,19492,0,0,"LINESTRING (2680842.326 1248448.377, 2680767.8...","(0, 19492, 0)"
1,7550201,NaN,residential,NaN,NaN,NaN,NaN,NaN,3,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,0.0,265.327,inf,265.327,inf,75310,0,0,"LINESTRING (2680842.326 1248448.377, 2680832.4...","(0, 75310, 0)"
2,7550202,NaN,residential,NaN,NaN,NaN,NaN,NaN,3,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,0.001,179.16211204283786,inf,87.78943490099056,inf,75311,0,0,"LINESTRING (2680842.326 1248448.377, 2680996.4...","(0, 75311, 0)"
3,7550203,NaN,secondary,NaN,NaN,NaN,NaN,NaN,6,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,0.006,188.7583535397085,inf,92.49159323445717,inf,641,1,0,"LINESTRING (2681100.025 1248300.320, 2680995.1...","(1, 641, 0)"
4,7550204,NaN,secondary,NaN,NaN,NaN,NaN,NaN,6,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,-0.01,142.5245332002142,inf,69.83702126810495,inf,2072,1,0,"LINESTRING (2681100.025 1248300.320, 2681177.1...","(1, 2072, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179146,7858414,NaN,primary,NaN,NaN,NaN,NaN,NaN,7,"""oneway""=>""yes"",""lanes""=>""2"",""sensors_backward...",...,0.0,577.8496357564222,inf,577.8496357564222,inf,75499,75500,0,"LINESTRING (2681547.559 1248996.114, 2681561.2...","(75500, 75499, 0)"
179147,7858415,NaN,primary,NaN,NaN,NaN,NaN,NaN,7,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,-0.0,577.8496357564222,inf,577.8496357564222,inf,40851,75501,0,"LINESTRING (2681535.315 1248971.038, 2681516.7...","(75501, 40851, 0)"
179148,7858416,NaN,primary,NaN,NaN,NaN,NaN,NaN,7,"""oneway""=>""yes"",""lanes""=>""2"",""sensors_backward...",...,0.0,577.8496357564222,inf,577.8496357564222,inf,75500,75501,0,"LINESTRING (2681535.315 1248971.038, 2681547.5...","(75501, 75500, 0)"
179149,7858417,NaN,secondary,NaN,NaN,NaN,NaN,NaN,6,"""oneway""=>""yes"",""lanes""=>""1"",""sensors_backward...",...,-0.001,107.38857471871358,inf,52.62040161216966,inf,70304,75502,0,"LINESTRING (2677332.791 1250090.020, 2677305.1...","(75502, 70304, 0)"


In [46]:
m = pd.merge(
    tt.reset_index(), osm_export[['osm_id', 'highway', 'uvk']],
    left_on='OSM_ID', right_on='osm_id', how='left'
)
m.drop(columns='osm_id', inplace=True)
m.set_index('uvk', inplace=True)
m

,OSM_ID,LinkId,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,...,car_22:00,bike_22:00,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,highway
uvk,,,,,,,,,,,,,,,,,,,,,
"(47357, 50430, 0)",7749021,3640,15.39,25.64,47.327088,8.381223,47.326734,8.380054,15.39,25.64,...,22.00,25.64,22.00,25.64,22.00,25.64,22.00,25.64,NaN,tertiary
"(34434, 25521, 0)",7701086,3638,22.48,22.48,47.499187,8.543563,47.499622,8.544628,22.48,22.48,...,28.00,22.48,22.48,22.48,22.48,22.48,22.48,22.48,NaN,residential
"(34435, 28848, 0)",7701087,3639,11.96,11.96,47.433009,8.668959,47.432571,8.669096,11.96,11.96,...,11.96,11.96,11.96,18.00,11.96,11.96,11.96,11.96,NaN,residential
"(34432, 34431, 0)",7701079,3630,40.03,40.03,47.490902,8.556413,47.492244,8.556424,40.03,40.03,...,46.00,40.03,40.03,40.03,40.03,40.03,46.00,40.03,NaN,residential
"(20718, 6230, 0)",7642172,3633,4.14,13.81,47.312737,8.913163,47.312457,8.913803,4.14,13.81,...,4.14,13.81,4.14,13.81,4.14,13.81,4.14,13.81,NaN,living_street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(32810, 41644, 0)",7694151,36256,30.34,30.34,47.536786,8.728047,47.536773,8.729726,30.34,30.34,...,30.34,30.34,30.34,30.34,30.34,30.34,30.34,30.34,NaN,residential
"(65460, 65453, 0)",7822078,132450,19.23,19.23,47.395599,8.401946,47.396158,8.401275,19.23,19.23,...,26.00,19.23,26.00,19.23,19.23,19.23,25.50,19.23,NaN,residential
"(45331, 45326, 0)",7740686,3629,46.18,46.18,47.349966,7.987588,47.351453,7.988677,46.18,46.18,...,53.00,46.18,46.18,46.18,53.00,46.18,53.00,46.18,NaN,residential


In [51]:
G.edges

OutMultiEdgeView([(0, 19492, 0), (0, 75310, 0), (0, 75311, 0), (1, 641, 0), (1, 2072, 0), (1, 10227, 0), (1, 75316, 0), (2, 646, 0), (2, 16557, 0), (2, 75320, 0), (2, 75328, 0), (3, 1998, 0), (3, 3516, 0), (3, 4016, 0), (3, 9110, 0), (4, 7054, 0), (4, 7060, 0), (4, 21500, 0), (5, 7054, 0), (5, 7065, 0), (6, 7062, 0), (6, 7065, 0), (6, 20534, 0), (7, 8, 0), (7, 20534, 0), (8, 22156, 0), (9, 10, 0), (9, 20542, 0), (9, 22156, 0), (10, 11, 0), (11, 16802, 0), (12, 16802, 0), (12, 21044, 0), (13, 19942, 0), (13, 19943, 0), (14, 4684, 0), (14, 14209, 0), (14, 19942, 0), (15, 7089, 0), (15, 19938, 0), (16, 17, 0), (16, 16367, 0), (17, 18, 0), (18, 19, 0), (18, 7084, 0), (19, 971, 0), (19, 15529, 0), (20, 4612, 0), (20, 4617, 0), (21, 4617, 0), (21, 4628, 0), (22, 23, 0), (22, 3063, 0), (22, 19929, 0), (23, 3062, 0), (24, 14127, 0), (24, 14128, 0), (24, 17574, 0), (25, 12047, 0), (25, 12595, 0), (25, 14129, 0), (26, 27, 0), (26, 241, 0), (26, 6328, 0), (27, 4638, 0), (28, 4631, 0), (28, 4639, 

In [50]:
i = 0
for uvk, data in G.edges.items():
    print(uvk)
    if uvk in tt.index:
        t = m[uvk][f'car_{TIME}']
        print(t)
    else:
        print('not')
    i += 1
    if i>20:
        break

(0, 19492, 0)
not
(0, 75310, 0)
not
(0, 75311, 0)
not
(1, 641, 0)
not
(1, 2072, 0)
not
(1, 10227, 0)
not
(1, 75316, 0)
not
(2, 646, 0)
not
(2, 16557, 0)
not
(2, 75320, 0)
not
(2, 75328, 0)
not
(3, 1998, 0)
not
(3, 3516, 0)
not
(3, 4016, 0)
not
(3, 9110, 0)
not
(4, 7054, 0)
not
(4, 7060, 0)
not
(4, 21500, 0)
not
(5, 7054, 0)
not
(5, 7065, 0)
not
(6, 7062, 0)
not


In [11]:
# convert the coordinates into geometries

transformer = pyproj.Transformer.from_crs(4326, 2056, always_xy=True)

tt['u'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['From_y'], row['From_x'])
    ),
    axis=1
)
tt['v'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['To_y'], row['To_x'])
    ),
    axis=1
)

tt['geometry'] = tt.apply(
    lambda row: shp.LineString([row['u'], row['v']]),
    axis=1
)

tt = gpd.GeoDataFrame(tt, crs=2056)

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,...,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,u,v,geometry
0,3640,7701054,22.34,44.68,47.523778,8.792639,47.522871,8.794674,22.34,44.68,...,27.10,44.68,29.00,44.68,24.62,44.68,NaN,POINT (2701971.2509862264 1264550.459269584),POINT (2702126.196330599 1264452.3290083027),"LINESTRING (2701971.251 1264550.459, 2702126.1..."
1,3638,7701052,13.91,27.81,47.523778,8.792639,47.524240,8.791260,13.91,27.81,...,20.00,27.81,13.91,27.81,20.00,27.81,NaN,POINT (2701971.2509862264 1264550.459269584),POINT (2701866.522171238 1264600.073411381),"LINESTRING (2701971.251 1264550.459, 2701866.5..."
2,3639,7701053,9.81,19.62,47.523778,8.792639,47.523084,8.792281,9.81,19.62,...,16.00,19.62,16.00,19.62,16.00,19.62,NaN,POINT (2701971.2509862264 1264550.459269584),POINT (2701945.5899760537 1264472.8407860398),"LINESTRING (2701971.251 1264550.459, 2701945.5..."
3,3630,7715687,5.86,11.73,47.534438,8.761990,47.534527,8.761355,5.86,11.73,...,12.00,15.00,12.00,15.00,12.00,15.00,NaN,POINT (2699643.28363578 1265696.1912974257),POINT (2699595.285331079 1265705.3345726437),"LINESTRING (2699643.284 1265696.191, 2699595.2..."
4,3631,7715688,4.39,8.79,47.534856,8.761385,47.534527,8.761355,4.39,8.79,...,11.00,8.79,11.00,12.00,11.00,8.79,NaN,POINT (2699596.957955538 1265741.9154586608),POINT (2699595.285331079 1265705.3345726437),"LINESTRING (2699596.958 1265741.915, 2699595.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143330,3629,7715686,9.62,19.24,47.534438,8.761990,47.534296,8.763034,9.62,19.24,...,14.06,23.00,9.62,23.00,16.00,19.24,NaN,POINT (2699643.28363578 1265696.1912974257),POINT (2699722.1511149006 1265681.7201363703),"LINESTRING (2699643.284 1265696.191, 2699722.1..."
143331,75829,7591240,3.14,10.48,47.276250,8.755897,47.276335,8.756461,3.14,10.48,...,10.00,10.48,10.00,14.00,10.00,10.48,NaN,POINT (2699666.474238684 1236986.3436005437),POINT (2699708.9396957774 1236996.4749395344),"LINESTRING (2699666.474 1236986.344, 2699708.9..."
143332,3628,7701043,17.95,35.91,47.522260,8.796916,47.523033,8.798388,17.95,35.91,...,17.95,35.91,17.95,35.91,18.00,35.91,NaN,POINT (2702296.2196253994 1264387.2857025354),POINT (2702405.547817276 1264475.1905265348),"LINESTRING (2702296.220 1264387.286, 2702405.5..."
143333,3627,7701042,11.98,23.97,47.522260,8.796916,47.521467,8.797539,11.98,23.97,...,11.98,23.97,11.98,23.97,11.98,23.97,NaN,POINT (2702296.2196253994 1264387.2857025354),POINT (2702344.6761799133 1264299.9677734694),"LINESTRING (2702296.220 1264387.286, 2702344.6..."


In [12]:
# reconstruct nodes and give them new IDs

nd = pd.concat([tt['u'], tt['v']]).reset_index().rename(columns={0: 'geometry'})
nd['osmid'] = pd.factorize(nd['geometry'])[0]
nd[['x', 'y']] = nd.apply(
    lambda row: (row['geometry'].x, row['geometry'].y),
    axis=1,
    result_type='expand'
)
nd.drop(columns=['index'], inplace=True)
nd.drop_duplicates(inplace=True)
nd.set_index('osmid', inplace=True)
nd = gpd.GeoDataFrame(nd, geometry='geometry', crs=2056)

nd

,geometry,x,y
osmid,,,
0,POINT (2701971.251 1264550.459),2.701971e+06,1.264550e+06
1,POINT (2699643.284 1265696.191),2.699643e+06,1.265696e+06
2,POINT (2699596.958 1265741.915),2.699597e+06,1.265742e+06
3,POINT (2702126.196 1264452.329),2.702126e+06,1.264452e+06
4,POINT (2638563.749 1242835.829),2.638564e+06,1.242836e+06
...,...,...,...
61804,POINT (2646040.731 1255217.002),2.646041e+06,1.255217e+06
61805,POINT (2671606.265 1255098.431),2.671606e+06,1.255098e+06
61806,POINT (2681171.862 1241293.762),2.681172e+06,1.241294e+06


In [13]:
# write the new node IDs into the edge table and create an index like in the street graph

tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='u', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['u', 'geometry_right'])
    .rename(columns={'osmid': 'u'}))
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='v', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['v', 'geometry_right'])
    .rename(columns={'osmid': 'v'}))

tt.drop(columns=['From_x', 'From_y', 'To_x', 'To_y'], inplace=True)

tt['uv'] = tt.apply(lambda row: (row['u'], row['v']), axis=1)
tt.set_index('uv', inplace=True)

tt['length'] = tt.apply(lambda row: row.geometry.length, axis=1)
# write the chosen travel time window as master travel time
tt['travel_time_s'] = tt[f'car_{TIME}']
tt['cost_']
#tt['highway'] = 'primary'
#tt['maxspeed'] = 40

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,car_0:00,bike_0:00,car_0:30,bike_0:30,car_1:00,bike_1:00,...,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,geometry,u,v,length,travel_time_s,maxspeed
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 3)",3640,7701054,22.34,44.68,22.34,44.68,22.34,44.68,22.34,44.68,...,44.68,24.62,44.68,NaN,"LINESTRING (2701971.251 1264550.459, 2702126.1...",0,3,183.405583,28.46,40
"(0, 6916)",3638,7701052,13.91,27.81,13.91,27.81,13.91,27.81,13.91,27.81,...,27.81,20.00,27.81,NaN,"LINESTRING (2701971.251 1264550.459, 2701866.5...",0,6916,115.886530,20.32,40
"(0, 4321)",3639,7701053,9.81,19.62,9.81,19.62,9.81,19.62,9.81,19.62,...,19.62,16.00,19.62,NaN,"LINESTRING (2701971.251 1264550.459, 2701945.5...",0,4321,81.750330,16.06,40
"(1, 57401)",3630,7715687,5.86,11.73,5.86,11.73,5.86,11.73,5.86,11.73,...,15.00,12.00,15.00,NaN,"LINESTRING (2699643.284 1265696.191, 2699595.2...",1,57401,48.861403,12.02,40
"(2, 57401)",3631,7715688,4.39,8.79,4.39,8.79,4.39,8.79,4.39,8.79,...,12.00,11.00,8.79,NaN,"LINESTRING (2699596.958 1265741.915, 2699595.2...",2,57401,36.619106,11.00,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(1, 11923)",3629,7715686,9.62,19.24,9.62,19.24,9.62,19.24,9.62,19.24,...,23.00,16.00,19.24,NaN,"LINESTRING (2699643.284 1265696.191, 2699722.1...",1,11923,80.184124,15.59,40
"(61652, 39215)",75829,7591240,3.14,10.48,3.14,10.48,3.14,10.48,3.14,10.48,...,14.00,10.00,10.48,NaN,"LINESTRING (2699666.474 1236986.344, 2699708.9...",61652,39215,43.657291,10.02,40
"(5, 30213)",3628,7701043,17.95,35.91,17.95,35.91,17.95,35.91,17.95,35.91,...,35.91,18.00,35.91,NaN,"LINESTRING (2702296.220 1264387.286, 2702405.5...",5,30213,140.285108,18.00,40


,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,car_0:00,bike_0:00,car_0:30,bike_0:30,car_1:00,bike_1:00,...,car_23:30,bike_23:30,Unnamed: 104,geometry,u,v,length,travel_time_s,maxspeed,highway
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 3)",3640,7701054,22.34,44.68,22.34,44.68,22.34,44.68,22.34,44.68,...,24.62,44.68,NaN,"LINESTRING (2701971.251 1264550.459, 2702126.1...",0,3,183.405583,28.46,40,residential
"(0, 6916)",3638,7701052,13.91,27.81,13.91,27.81,13.91,27.81,13.91,27.81,...,20.00,27.81,NaN,"LINESTRING (2701971.251 1264550.459, 2701866.5...",0,6916,115.886530,20.32,40,residential
"(0, 4321)",3639,7701053,9.81,19.62,9.81,19.62,9.81,19.62,9.81,19.62,...,16.00,19.62,NaN,"LINESTRING (2701971.251 1264550.459, 2701945.5...",0,4321,81.750330,16.06,40,residential
"(1, 57401)",3630,7715687,5.86,11.73,5.86,11.73,5.86,11.73,5.86,11.73,...,12.00,15.00,NaN,"LINESTRING (2699643.284 1265696.191, 2699595.2...",1,57401,48.861403,12.02,40,secondary
"(2, 57401)",3631,7715688,4.39,8.79,4.39,8.79,4.39,8.79,4.39,8.79,...,11.00,8.79,NaN,"LINESTRING (2699596.958 1265741.915, 2699595.2...",2,57401,36.619106,11.00,40,residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(1, 11923)",3629,7715686,9.62,19.24,9.62,19.24,9.62,19.24,9.62,19.24,...,16.00,19.24,NaN,"LINESTRING (2699643.284 1265696.191, 2699722.1...",1,11923,80.184124,15.59,40,secondary
"(61652, 39215)",75829,7591240,3.14,10.48,3.14,10.48,3.14,10.48,3.14,10.48,...,10.00,10.48,NaN,"LINESTRING (2699666.474 1236986.344, 2699708.9...",61652,39215,43.657291,10.02,40,tertiary
"(5, 30213)",3628,7701043,17.95,35.91,17.95,35.91,17.95,35.91,17.95,35.91,...,18.00,35.91,NaN,"LINESTRING (2702296.220 1264387.286, 2702405.5...",5,30213,140.285108,18.00,40,residential


In [15]:
# make all links oneway
tt['oneway'] = 'yes'

In [17]:
# create a new street graph
G = snman.street_graph.street_graph_from_gdf(nd, tt)

In [11]:
snman.io.export_osm_xml(
    G,
    os.path.join(outputs_path, f'tt_car_{STATE}.osm'),
    tags=['LinkId', 'Freespeed', 'maxspeed', 'travel_time_s', 'length', 'highway', 'oneway'],
    uv_tags=True,
    overwrite_highway='primary',
    dont_overwrite_highway=['path'],
    set_maxspeed_by_cost='travel_time_s',
    raw_graph=True,
    floor_maxspeed=0.1,
    ceil_maxspeed=120
)

snman.io.osm_to_pbf(os.path.join(outputs_path, f'tt_car_{STATE}.osm'))

In [18]:
if 1:
    snman.io.export_street_graph(
        G,
        os.path.join(outputs_path, f'tt_{STATE}_edges.gzip'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gzip'),
        crs=CRS_for_export
    )